In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import string 
import re
from scipy import spatial
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk import pos_tag
import os
from sklearn.naive_bayes import GaussianNB
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stopword = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install transformers==4.1.1 sentencepiece==0.1.94
!pip install mosestokenizer==1.1.0

     |████████████████████████████████| 1.5 MB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 59.4 MB/s 
     |████████████████████████████████| 895 kB 60.1 MB/s 
     |████████████████████████████████| 2.9 MB 31.1 MB/s 
  Created wheel for mosestokenizer: filename=mosestokenizer-1.1.0-py3-none-any.whl size=49117 sha256=7798396e76926dbd86c9b5c05f7a3a02f493c1ee5efed4e44dc4a09079863948
  Stored in directory: /root/.cache/pip/wheels/a7/31/94/fef279382208e85a65c1a7f5c4d0020115477b0af74f296b57
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3354 sha256=e3e25c562330fe1289fa406242f6f9a39d63e11a5ac107edf8eb46514529e65a
  Stored in directory: /root/.cache/pip/wheels/c5/4f/33/54741ffe08e38ececb1d28068a153729b4fe820bafa0a0691f
  Created wheel for uctools: filename=uctools-1.3.0-py3-none-any.whl size=6163 sha256=4db1a9cfd1d30992cc61a7f86364c9cc164b0b4e6655e9fae6587413b11c39b2
  Stored in directory: /root/.cache/pip/wheels/fb/44/e9/914cf8fa71f0141f9314f86

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

In [ ]:
target_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
target_model = MarianMTModel.from_pretrained(target_model_name)

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/799k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/265 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

Some weights of MarianMTModel were not initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-ROMANCE and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
en_tokenizer = MarianTokenizer.from_pretrained(en_model_name)
en_model = MarianMTModel.from_pretrained(en_model_name)

Downloading:   0%|          | 0.00/800k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/265 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

Some weights of MarianMTModel were not initialized from the model checkpoint at Helsinki-NLP/opus-mt-ROMANCE-en and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def translate(texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    template = lambda text: f"{text}" if language == "en" else f">>{language}<< {text}"
    src_texts = [template(text) for text in texts]
    src_texts = np.array(src_texts)
    # Tokenize the texts
    encoded = tokenizer.prepare_seq2seq_batch(src_texts,return_tensors='pt')
    
    # Generate translation using model
    translated = model.generate(**encoded)

    # Convert the generated tokens indices back into text
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return translated_texts

In [ ]:
def back_translate(texts, source_lang="en", target_lang="fr"):
    # Translate from source to target language
    fr_texts = translate(texts, target_model, target_tokenizer, 
                         language=target_lang)

    # Translate from target language back to source language
    back_translated_texts = translate(fr_texts, en_model, en_tokenizer, 
                                      language=source_lang)
    
    return back_translated_texts

In [ ]:
en_texts = ['This is so cool', 'I hated the food', 'They were very helpful']
aug_texts = back_translate(np.array(en_texts), source_lang="en", target_lang="fr")
print(aug_texts)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1065: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


["It's so cool.", 'I hated food.', "They've been very helpful."]


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
 df=pd.read_csv('/content/drive/MyDrive/PRISM/augmented_la.csv')

In [ ]:
df.head()

,Unnamed: 0,review
0,0,['This short meter that inspired the next long...
1,1,"['Okay, night man August 18, 2004 I distinctly..."
2,2,"[""This is a game in impulse I'm buying a walma..."
3,3,"['No, hilarious 70 did for TV horror clinker i..."
4,4,['It is easy to say in this film exactly what ...


In [ ]:
df['review'][0]

'[\'This short meter that inspired the next longitude - Spatula Madness - is hilarious that it contends against similar postcards yielding multiple authorities\', \'Short film of the star Eduardo Spatula, who after his work united in luxury against the spoon of evil.\', \'This premise allows one to be content in the usual manner, and is not present in the residue.\', \'Five-fifth time of tiny film is absorbed from errata comedy and tiny musical nuntiums\', "Regrettably not super women\'s women", \'The place which seven has no time to display\', \'Then he followed him alone on the high budget of the Hollywood film.\', \'This is the film of the custody of the one time.\', \'Tell him what he is and do not wait for the story to be told in depth.\']'

In [ ]:
for i in range(len(df['review'])):
  str=""
  for x in df['review'][i]:
    if x!='\'' and x!='[' and x!= ']':
       str=str+x
  df['review'][i]=str

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df['review'][0]

'This short meter that inspired the next longitude - Spatula Madness - is hilarious that it contends against similar postcards yielding multiple authorities, Short film of the star Eduardo Spatula, who after his work united in luxury against the spoon of evil., This premise allows one to be content in the usual manner, and is not present in the residue., Five-fifth time of tiny film is absorbed from errata comedy and tiny musical nuntiums, "Regrettably not super womens women", The place which seven has no time to display, Then he followed him alone on the high budget of the Hollywood film., This is the film of the custody of the one time., Tell him what he is and do not wait for the story to be told in depth.'

In [ ]:
for i in range(len(df['review'])):
  str=""
  for j in range(len(df['review'][i])):
    if df['review'][i][j] == ',' and df['review'][i][j-1]=='.':
      pass
    else:
      str=str+df['review'][i][j]
  df['review'][i]=str

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df['review'][0]

'This short meter that inspired the next longitude - Spatula Madness - is hilarious that it contends against similar postcards yielding multiple authorities, Short film of the star Eduardo Spatula, who after his work united in luxury against the spoon of evil. This premise allows one to be content in the usual manner, and is not present in the residue. Five-fifth time of tiny film is absorbed from errata comedy and tiny musical nuntiums, "Regrettably not super womens women", The place which seven has no time to display, Then he followed him alone on the high budget of the Hollywood film. This is the film of the custody of the one time. Tell him what he is and do not wait for the story to be told in depth.'

In [ ]:
X_augmented=list(df['review'])

In [ ]:
df_new=pd.read_csv('/content/Augmented_text(1) .csv')

In [ ]:
df_new.head()

,Unnamed: 0,Text,Augmented Text
0,0,One of the other reviewers has mentioned that ...,One of the other reviewers mentioned what afte...
1,1,A wonderful little production. <br /><br />The...,Wonderful production.The technique of filming ...
2,2,I thought this was a wonderful way to spend ti...,"I thought that this was a wonder, that it migh..."
3,3,Basically there's a family where a little boy ...,"Basically, the family is where none (Jake) thi..."
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","Petter Mattei's ""Love in Time Money"" is the vi..."


In [ ]:
X_augmented_first=list(df_new['Augmented Text'])

In [ ]:
augmented=X_augmented_first+X_augmented

In [ ]:
len(augmented)

200

In [ ]:
for i in range(100):
  X[i]=X[i].split('.')

In [ ]:
for i in range(100):
  X_t=[]
  for s in X[i]:
    if s != '':
      X_t.append(s)
  X[i]=X_t


In [ ]:
X[0]

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked",
 ' They are right, as this is exactly what happened with me',
 '<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO',
 ' Trust me, this is not a show for the faint hearted or timid',
 ' This show pulls no punches with regards to drugs, sex or violence',
 ' Its is hardcore, in the classic use of the word',
 '<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary',
 ' It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda',
 ' Em City is home to many',
 'Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more',
 'so scuffles, death stares, dodgy dealings and shady agreements are never far away',
 "<br /><br />I would say the

In [ ]:
# X=[]
# for text in X_text:
#   text=text.split(".")
#   #print(len(text))
#   X.append(text)

In [ ]:

# aug_texts = back_translate(np.array(X_text[0]), source_lang="en", target_lang="es")
# print(aug_texts)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1065: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


['One of the other critics has mentioned that after seeing only one episode of 1 Oz you will be hooked', "You're right, because this is exactly what happened to me.", 'The first thing that surprised me about Oz was his brutality and unbreakable scenes of violence, which began from the word GO', "Trust me, this isn't a show for the weak of heart or shy", "This program doesn't knock on drugs, sex or violence", 'It is hardcore, in the classic use of the word', "<br /><br />It's called OZ as that's the nickname given to Oswald's Maximum Security State Penitentiary", 'It focuses mainly on Emerald City, an experimental section of the prison where all cells have glass fronts and face in, so privacy is not a priority on the agenda', 'Em City is home to many', 'Aryans, Muslims, gangsters, Latins, Christians, Italians, Irish and more', 'So the skirmishes, the looks of death, the bad relations and the gloomy agreements are never far away.', 'I would say that the main attraction of the show is due

In [ ]:
X_augmented=[]
i=1
for text in X_text:
  print(i)
  aug_texts = back_translate(np.array(text), source_lang="en", target_lang="la")
  X_augmented.append(aug_texts)
  print(aug_texts)
  i=i+1

1


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1065: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


['This short meter that inspired the next longitude - Spatula Madness - is hilarious that it contends against similar postcards yielding multiple authorities', 'Short film of the star Eduardo Spatula, who after his work united in luxury against the spoon of evil.', 'This premise allows one to be content in the usual manner, and is not present in the residue.', 'Five-fifth time of tiny film is absorbed from errata comedy and tiny musical nuntiums', "Regrettably not super women's women", 'The place which seven has no time to display', 'Then he followed him alone on the high budget of the Hollywood film.', 'This is the film of the custody of the one time.', 'Tell him what he is and do not wait for the story to be told in depth.']
2
['Okay, night man August 18, 2004 I distinctly displace adversum Dominus', 'Van Bebble in prospection filmum Familia Manson in Trio Penny in Chicago which part of Chicago Underground Film Festival', 'This is what I say about her.', 'First of all, film is obviou

In [ ]:
df_new=pd.DataFrame()

In [ ]:
df_new['review']=X_augmented

In [ ]:
df_new.to_csv("/content/drive/MyDrive/PRISM/augmented_la_0to100.csv")

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/PRISM/IMDB Dataset.csv")

In [ ]:
X_text=df['review']

In [ ]:
X_text=X_text[:200]

In [ ]:
len(augmented)

200

In [ ]:
X_augment=[]
y_train=[]
y_augmented_train=[]
for i in range(200):
  X_augment.append(X_text[i])
  X_augment.append(augmented[i])
  if df['sentiment'][i]=='positive':
    y_train.append(1)
    y_augmented_train.append(1)
    y_augmented_train.append(1)
  else:
    y_train.append(0)
    y_augmented_train.append(0)
    y_augmented_train.append(0)

In [ ]:
len(X_augment)

400

In [ ]:
en_stopwords=set(stopwords.words('english'))
wn = nltk.WordNetLemmatizer()

In [ ]:
def getCleanedReview(review):
    review=review.lower()
    text_lc = "".join([word.lower() for word in review if word not in string.punctuation])
    #print(text_lc)
    text_rc=re.sub('[0-9]+',' ',text_lc)
    #tokenize
    tokens=word_tokenize(text_rc,language='english')
    #print(tokens)
    new_tokens=[token for token in tokens if token not in en_stopwords]
    #print(new_tokens)
    lemmatized_tokens=[wn.lemmatize(token) for token in new_tokens]
    return lemmatized_tokens

In [ ]:
original=[]
y_augmented=[]
X_train=[]
X_augmented_train=[]
y=df['sentiment']

In [ ]:
for i in range(len(X_text)):
    X_train.append(getCleanedReview(X_text[i]))
    if y[i] == 'positive':
        original.append(1)
        y_augmented.append(1)
        y_augmented.append(1)
    else:
        original.append(0)
        y_augmented.append(0)
        y_augmented.append(0)

In [ ]:
for i in range(len(X_augment)):
    X_augmented_train.append(getCleanedReview(X_augment[i]))

In [ ]:
len(X_augmented_train)

400

In [ ]:
df_new=[]
df_new=pd.DataFrame(df_new)

In [ ]:
df_new["cleaned_review"]=X_train
X_train=df_new["cleaned_review"]

In [ ]:
df_new_=[]
df_new_=pd.DataFrame(df_new_)
df_new_["cleaned_augmented_review"]=X_augmented_train
X_augmented_train=df_new_["cleaned_augmented_review"]

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import sklearn.metrics

In [ ]:
X_train.dropna()
cvt = CountVectorizer(analyzer=getCleanedReview,max_features=6000)
#cvt2 = CountVectorizer(analyzer=getCleanedReview,max_features=6000)
X_v = cvt.fit_transform(X_train.apply(lambda x: np.str_(x)))
X_augmented_v=cvt.fit_transform(X_augmented_train.apply(lambda x: np.str_(x)))

In [ ]:
y_train=np.array(original)
y_augmented=np.array(y_augmented)

In [ ]:
gnb=GaussianNB()
X_v=X_v.toarray()
X_augmented_v=X_augmented_v.toarray()
gnb.fit(X_v,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
gnb2=GaussianNB()
gnb2.fit(X_augmented_v,y_augmented)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
X_test=df['review'].iloc[25000:25600]
y_test=df['sentiment'].iloc[25000:25600]

In [ ]:
X_test=X_test.values.tolist()

In [ ]:
X_test_new=[]
y_test_new=[]

In [ ]:
for i in range(len(X_test)):
    X_test_new.append(getCleanedReview(X_test[i]))
    if y_test[25000+i] == 'positive':
        y_test_new.append(1)
    else:
        y_test_new.append(0)

In [ ]:
df_test=pd.DataFrame()

In [ ]:

df_test["cleaned_test_review"]=X_test_new
X_test=df_test["cleaned_test_review"]

In [ ]:
X_test.dropna()
cvt1 = CountVectorizer(analyzer=getCleanedReview,max_features=6000)
#cvta= CountVectorizer(analyzer=getCleanedReview,max_features=6000)
X_test_v = cvt1.fit_transform(X_test.apply(lambda x: np.str_(x)))
#X_test_v_a = cvta.fit_transform(X_test.apply(lambda x: np.str_(x)))

In [ ]:
y_test=np.array(y_test_new)

In [ ]:
X_test_v=X_test_v.toarray()
#X_test_v_a=X_test_v_a.toarray()

In [ ]:
gnb.score(X_test_v,y_test) #original accuracy

0.49

In [ ]:
gnb2.score(X_test_v,y_test) #augmented accuracy

0.5433333333333333